In [1]:
# MTA turnstile data for all stations
# week: November 10-16, 2018
# extracting: M-F, AM, PM
# average riders per stop for AM and PM, entering and exiting

In [2]:
# import Dependencies
import pandas as pd
import numpy as np

In [3]:
# data from MTA: http://web.mta.info/developers/turnstile.html
csv_path = "turnstile_181117.csv"
ts_11_2018_all = pd.read_csv(csv_path, encoding="utf-8")

In [4]:
# format date and time for Pandas
ts_11_2018_all["DATE"] = pd.to_datetime(ts_11_2018_all["DATE"])
ts_11_2018_all["TIME"] = pd.to_datetime(ts_11_2018_all["TIME"])

In [5]:
# add 'day of week' column - prep to remove Saturdy and Sunday
ts_11_2018_all["WEEKDAY"] = ts_11_2018_all["DATE"].dt.day_name()

In [6]:
# create new dataframe, as to not completely lose weekend data
ts_11_2018 = ts_11_2018_all

In [7]:
# use .loc to remove Saturday annd Sunday
ts_11_2018 = ts_11_2018.loc[(ts_11_2018["WEEKDAY"]!= "Saturday") & (ts_11_2018["WEEKDAY"]!= "Sunday"), : ]
ts_11_2018.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,WEEKDAY
12,A002,R051,02-00-00,59 ST,NQR456W,BMT,2018-11-12,2019-08-27 03:00:00,REGULAR,6831427,2316424,Monday
13,A002,R051,02-00-00,59 ST,NQR456W,BMT,2018-11-12,2019-08-27 07:00:00,REGULAR,6831438,2316474,Monday
14,A002,R051,02-00-00,59 ST,NQR456W,BMT,2018-11-12,2019-08-27 11:00:00,REGULAR,6831570,2316713,Monday
15,A002,R051,02-00-00,59 ST,NQR456W,BMT,2018-11-12,2019-08-27 15:00:00,REGULAR,6831812,2316785,Monday
16,A002,R051,02-00-00,59 ST,NQR456W,BMT,2018-11-12,2019-08-27 19:00:00,REGULAR,6832592,2316857,Monday


In [8]:
# check for missing station data
# ts_11_2018[ts_11_2018['STATION'] == '28 ST']

In [9]:
# check column titles because there is a problem with EXIT
ts_11_2018.columns

Index(['C/A', 'UNIT', 'SCP', 'STATION', 'LINENAME', 'DIVISION', 'DATE', 'TIME',
       'DESC', 'ENTRIES',
       'EXITS                                                               ',
       'WEEKDAY'],
      dtype='object')

In [10]:
# remane columns: entries / exits, as these are cummulative
ts_11_2018 = ts_11_2018.rename(columns={'ENTRIES': 'TOT. ENTRIES'})
ts_11_2018 = ts_11_2018.rename(columns={'EXITS                                                               '
                                                      : 'TOT. EXITS'})
ts_11_2018.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,TOT. ENTRIES,TOT. EXITS,WEEKDAY
12,A002,R051,02-00-00,59 ST,NQR456W,BMT,2018-11-12,2019-08-27 03:00:00,REGULAR,6831427,2316424,Monday
13,A002,R051,02-00-00,59 ST,NQR456W,BMT,2018-11-12,2019-08-27 07:00:00,REGULAR,6831438,2316474,Monday
14,A002,R051,02-00-00,59 ST,NQR456W,BMT,2018-11-12,2019-08-27 11:00:00,REGULAR,6831570,2316713,Monday
15,A002,R051,02-00-00,59 ST,NQR456W,BMT,2018-11-12,2019-08-27 15:00:00,REGULAR,6831812,2316785,Monday
16,A002,R051,02-00-00,59 ST,NQR456W,BMT,2018-11-12,2019-08-27 19:00:00,REGULAR,6832592,2316857,Monday


In [11]:
# enter/exit are cummulative, need to subtract from previous
# copying ENTERS, shifting down a row, then substracting
# using abs to give us a positive number
# create new column to hold entries
ts_11_2018['ENTERS'] = abs(ts_11_2018['TOT. ENTRIES'].shift(1) - ts_11_2018['TOT. ENTRIES'])

In [12]:
# need to repeat steps for exits
ts_11_2018['EXITS'] = abs(ts_11_2018['TOT. EXITS'].shift(1) - ts_11_2018['TOT. EXITS'])
ts_11_2018.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,TOT. ENTRIES,TOT. EXITS,WEEKDAY,ENTERS,EXITS
12,A002,R051,02-00-00,59 ST,NQR456W,BMT,2018-11-12,2019-08-27 03:00:00,REGULAR,6831427,2316424,Monday,NaN,NaN
13,A002,R051,02-00-00,59 ST,NQR456W,BMT,2018-11-12,2019-08-27 07:00:00,REGULAR,6831438,2316474,Monday,11.0,50.0
14,A002,R051,02-00-00,59 ST,NQR456W,BMT,2018-11-12,2019-08-27 11:00:00,REGULAR,6831570,2316713,Monday,132.0,239.0
15,A002,R051,02-00-00,59 ST,NQR456W,BMT,2018-11-12,2019-08-27 15:00:00,REGULAR,6831812,2316785,Monday,242.0,72.0
16,A002,R051,02-00-00,59 ST,NQR456W,BMT,2018-11-12,2019-08-27 19:00:00,REGULAR,6832592,2316857,Monday,780.0,72.0


In [13]:
# new data frame = focus on morning commute: DAMN YOU, MTA!
ts_11_2018_am = ts_11_2018

In [14]:
# CORRECT way to isolate mornings commuters!
ts_11_2018_am = ts_11_2018_am.loc[(ts_11_2018_am["TIME"].dt.hour >= 5) & (ts_11_2018_am["TIME"].dt.hour <= 9), :]
ts_11_2018_am.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,TOT. ENTRIES,TOT. EXITS,WEEKDAY,ENTERS,EXITS
13,A002,R051,02-00-00,59 ST,NQR456W,BMT,2018-11-12,2019-08-27 07:00:00,REGULAR,6831438,2316474,Monday,11.0,50.0
19,A002,R051,02-00-00,59 ST,NQR456W,BMT,2018-11-13,2019-08-27 07:00:00,REGULAR,6832901,2316944,Tuesday,16.0,46.0
25,A002,R051,02-00-00,59 ST,NQR456W,BMT,2018-11-14,2019-08-27 07:00:00,REGULAR,6834427,2317507,Wednesday,18.0,60.0
39,A002,R051,02-00-01,59 ST,NQR456W,BMT,2018-11-12,2019-08-27 07:00:00,REGULAR,6104322,1367964,Monday,11.0,19.0
45,A002,R051,02-00-01,59 ST,NQR456W,BMT,2018-11-13,2019-08-27 07:00:00,REGULAR,6105426,1368189,Tuesday,19.0,18.0


In [14]:
# # WAY MORE COMPLICATED WAY TO ISOLATE MORNING COMMUTE!
# # extract AM data
# ts_11_2018_am = ts_11_2018_am.loc[(ts_11_2018_am["TIME"]=="05:00:00") | (ts_11_2018_am["TIME"]=="09:00:00") |
#                                   (ts_11_2018_am["TIME"]=="05:15:28") | (ts_11_2018_am["TIME"]=="05:30:00") |
#                                   (ts_11_2018_am["TIME"]=="06:38:21") | (ts_11_2018_am["TIME"]=="07:00:00") | 
#                                   (ts_11_2018_am["TIME"]=="07:22:00") | (ts_11_2018_am["TIME"]=="07:25:28") |
#                                   (ts_11_2018_am["TIME"]=="07:30:00") | (ts_11_2018_am["TIME"]=="07:38:52") |
#                                   (ts_11_2018_am["TIME"]=="07:50:33") | (ts_11_2018_am["TIME"]=="07:52:47") |
#                                   (ts_11_2018_am["TIME"]=="07:53:17") | (ts_11_2018_am["TIME"]=="08:00:00") |
#                                   (ts_11_2018_am["TIME"]=="08:02:06") | (ts_11_2018_am["TIME"]=="08:02:10") |
#                                   (ts_11_2018_am["TIME"]=="08:02:16") | (ts_11_2018_am["TIME"]=="08:02:19") |
#                                   (ts_11_2018_am["TIME"]=="08:02:23") | (ts_11_2018_am["TIME"]=="08:02:25") |
#                                   (ts_11_2018_am["TIME"]=="08:02:28") | (ts_11_2018_am["TIME"]=="08:02:35") |
#                                   (ts_11_2018_am["TIME"]=="08:02:39") | (ts_11_2018_am["TIME"]=="08:02:43") |
#                                   (ts_11_2018_am["TIME"]=="08:02:46") | (ts_11_2018_am["TIME"]=="08:02:51") |
#                                   (ts_11_2018_am["TIME"]=="08:02:54") | (ts_11_2018_am["TIME"]=="08:02:59") |
#                                   (ts_11_2018_am["TIME"]=="08:03:06") | (ts_11_2018_am["TIME"]=="08:03:15") |
#                                   (ts_11_2018_am["TIME"]=="08:03:21") | (ts_11_2018_am["TIME"]=="06:00:00") |
#                                   (ts_11_2018_am["TIME"]=="08:03:27") | (ts_11_2018_am["TIME"]=="08:03:32") |
#                                   (ts_11_2018_am["TIME"]=="08:03:35") | (ts_11_2018_am["TIME"]=="08:03:38") |
#                                   (ts_11_2018_am["TIME"]=="08:08:31") | (ts_11_2018_am["TIME"]=="08:21:07") |
#                                   (ts_11_2018_am["TIME"]=="08:22:45") | (ts_11_2018_am["TIME"]=="08:23:47") |
#                                   (ts_11_2018_am["TIME"]=="08:25:07") | (ts_11_2018_am["TIME"]=="08:26:47") |
#                                   (ts_11_2018_am["TIME"]=="08:28:35") | (ts_11_2018_am["TIME"]=="08:28:39") |
#                                   (ts_11_2018_am["TIME"]=="08:28:44") | (ts_11_2018_am["TIME"]=="08:29:08") |
#                                   (ts_11_2018_am["TIME"]=="08:30:44") | (ts_11_2018_am["TIME"]=="08:32:04") |
#                                   (ts_11_2018_am["TIME"]=="08:33:24") | (ts_11_2018_am["TIME"]=="08:56:08") |
#                                   (ts_11_2018_am["TIME"]=="09:00:00") | (ts_11_2018_am["TIME"]=="09:01:02") |
#                                   (ts_11_2018_am["TIME"]=="09:10:42") | (ts_11_2018_am["TIME"]=="09:11:13") |
#                                   (ts_11_2018_am["TIME"]=="09:11:16") | (ts_11_2018_am["TIME"]=="09:11:21") |
#                                   (ts_11_2018_am["TIME"]=="09:15:28") | (ts_11_2018_am["TIME"]=="09:17:12") |
#                                   (ts_11_2018_am["TIME"]=="09:17:16") | (ts_11_2018_am["TIME"]=="09:17:21") |
#                                   (ts_11_2018_am["TIME"]=="09:17:40") | (ts_11_2018_am["TIME"]=="09:17:46") |
#                                   (ts_11_2018_am["TIME"]=="09:17:54") | (ts_11_2018_am["TIME"]=="09:18:08") |
#                                   (ts_11_2018_am["TIME"]=="09:18:13") | (ts_11_2018_am["TIME"]=="09:18:17") |
#                                   (ts_11_2018_am["TIME"]=="09:18:21") | (ts_11_2018_am["TIME"]=="09:18:24") |
#                                   (ts_11_2018_am["TIME"]=="09:18:27") | (ts_11_2018_am["TIME"]=="09:18:31") |
#                                   (ts_11_2018_am["TIME"]=="09:18:35") | (ts_11_2018_am["TIME"]=="09:18:38") |
#                                   (ts_11_2018_am["TIME"]=="09:18:42") | (ts_11_2018_am["TIME"]=="09:18:46") |
#                                   (ts_11_2018_am["TIME"]=="09:18:48") | (ts_11_2018_am["TIME"]=="09:19:00") |
#                                   (ts_11_2018_am["TIME"]=="09:19:03") | (ts_11_2018_am["TIME"]=="09:19:15") |
#                                   (ts_11_2018_am["TIME"]=="09:19:07") | (ts_11_2018_am["TIME"]=="09:19:11") |
#                                   (ts_11_2018_am["TIME"]=="09:19:12") | (ts_11_2018_am["TIME"]=="09:19:17") |
#                                   (ts_11_2018_am["TIME"]=="09:19:21") | (ts_11_2018_am["TIME"]=="09:19:22") |
#                                   (ts_11_2018_am["TIME"]=="09:19:27") | (ts_11_2018_am["TIME"]=="09:19:31") |
#                                   (ts_11_2018_am["TIME"]=="09:19:38") | (ts_11_2018_am["TIME"]=="09:19:39") |
#                                   (ts_11_2018_am["TIME"]=="09:19:43") | (ts_11_2018_am["TIME"]=="09:19:45") |
#                                   (ts_11_2018_am["TIME"]=="09:19:48") | (ts_11_2018_am["TIME"]=="09:19:56") |
#                                   (ts_11_2018_am["TIME"]=="09:20:00") | (ts_11_2018_am["TIME"]=="09:20:15") |
#                                   (ts_11_2018_am["TIME"]=="09:20:31") | (ts_11_2018_am["TIME"]=="09:20:34") |
#                                   (ts_11_2018_am["TIME"]=="09:20:37") | (ts_11_2018_am["TIME"]=="09:20:51") |
#                                   (ts_11_2018_am["TIME"]=="09:21:03") | (ts_11_2018_am["TIME"]=="09:21:06") |
#                                   (ts_11_2018_am["TIME"]=="09:21:11") | (ts_11_2018_am["TIME"]=="09:21:13") |
#                                   (ts_11_2018_am["TIME"]=="09:21:14") | (ts_11_2018_am["TIME"]=="09:21:17") |                               
#                                   (ts_11_2018_am["TIME"]=="09:21:23") | (ts_11_2018_am["TIME"]=="09:21:26") |                               
#                                   (ts_11_2018_am["TIME"]=="09:21:29") | (ts_11_2018_am["TIME"]=="09:21:32") |                               
#                                   (ts_11_2018_am["TIME"]=="09:21:37") | (ts_11_2018_am["TIME"]=="09:21:40") |                               
#                                   (ts_11_2018_am["TIME"]=="09:21:46") | (ts_11_2018_am["TIME"]=="09:21:53") |                               
#                                   (ts_11_2018_am["TIME"]=="09:22:02") | (ts_11_2018_am["TIME"]=="09:22:05") |                               
#                                   (ts_11_2018_am["TIME"]=="09:22:12") | (ts_11_2018_am["TIME"]=="09:23:10") |                               
#                                   (ts_11_2018_am["TIME"]=="09:29:18") | (ts_11_2018_am["TIME"]=="09:30:00") , :]
# ts_11_2018_am.head()

In [15]:
# clean up data frame
ts_11_2018_am = ts_11_2018_am.drop(['C/A','UNIT','SCP','DIVISION','DESC','TOT. ENTRIES','TOT. EXITS'], axis=1)

In [16]:
ts_11_2018_am.head()

,STATION,LINENAME,DATE,TIME,WEEKDAY,ENTERS,EXITS
13,59 ST,NQR456W,2018-11-12,2019-08-27 07:00:00,Monday,11.0,50.0
19,59 ST,NQR456W,2018-11-13,2019-08-27 07:00:00,Tuesday,16.0,46.0
25,59 ST,NQR456W,2018-11-14,2019-08-27 07:00:00,Wednesday,18.0,60.0
39,59 ST,NQR456W,2018-11-12,2019-08-27 07:00:00,Monday,11.0,19.0
45,59 ST,NQR456W,2018-11-13,2019-08-27 07:00:00,Tuesday,19.0,18.0


In [17]:
# REMOVE ENTER COUNTS WITH 6 AND OVER DIGITS!
ts_11_2018_am = ts_11_2018_am.loc[ts_11_2018_am['ENTERS'] <= 99999, :]

In [18]:
# REMOVE EXIT COUNTS WITH 6 AND OVER DIGITS!
ts_11_2018_am = ts_11_2018_am.loc[ts_11_2018_am['EXITS'] <= 99999, :]
ts_11_2018_am.head()

,STATION,LINENAME,DATE,TIME,WEEKDAY,ENTERS,EXITS
13,59 ST,NQR456W,2018-11-12,2019-08-27 07:00:00,Monday,11.0,50.0
19,59 ST,NQR456W,2018-11-13,2019-08-27 07:00:00,Tuesday,16.0,46.0
25,59 ST,NQR456W,2018-11-14,2019-08-27 07:00:00,Wednesday,18.0,60.0
39,59 ST,NQR456W,2018-11-12,2019-08-27 07:00:00,Monday,11.0,19.0
45,59 ST,NQR456W,2018-11-13,2019-08-27 07:00:00,Tuesday,19.0,18.0


In [43]:
# CHECK KEW GARDENS!
kew_gardens = ts_11_2018_am[ts_11_2018_am['STATION'] == 'KEW GARDENS']
kew_gardens.head()

,STATION,LINENAME,DATE,TIME,WEEKDAY,ENTERS,EXITS
84659,KEW GARDENS,EF,2018-11-12,2019-08-27 05:30:00,Monday,54.0,13.0
84660,KEW GARDENS,EF,2018-11-12,2019-08-27 09:30:00,Monday,822.0,243.0
84665,KEW GARDENS,EF,2018-11-13,2019-08-27 05:30:00,Tuesday,51.0,8.0
84666,KEW GARDENS,EF,2018-11-13,2019-08-27 09:30:00,Tuesday,1088.0,141.0
84671,KEW GARDENS,EF,2018-11-14,2019-08-27 05:30:00,Wednesday,62.0,16.0


In [ ]:
# save as csv
kew_gardens.to_csv('kew_gardens.csv')

In [46]:
# COMPARE KEW GARDENS WITH LARGER STATION!
tsx = ts_11_2018_am[ts_11_2018_am['STATION'] == '42 ST-PORT AUTH']
tsx.head()

,STATION,LINENAME,DATE,TIME,WEEKDAY,ENTERS,EXITS
49148,42 ST-PORT AUTH,ACENGRS1237W,2018-11-12,2019-08-27 05:00:00,Monday,36.0,48.0
49149,42 ST-PORT AUTH,ACENGRS1237W,2018-11-12,2019-08-27 09:00:00,Monday,70.0,582.0
49154,42 ST-PORT AUTH,ACENGRS1237W,2018-11-13,2019-08-27 05:00:00,Tuesday,33.0,50.0
49155,42 ST-PORT AUTH,ACENGRS1237W,2018-11-13,2019-08-27 09:00:00,Tuesday,99.0,718.0
49160,42 ST-PORT AUTH,ACENGRS1237W,2018-11-14,2019-08-27 05:00:00,Wednesday,37.0,56.0


In [47]:
# save as csv
tsx.to_csv('tsx.csv')

In [19]:
# groupby station name and get total (not average)
ts_11_2018_am_tot = ts_11_2018_am.groupby(['STATION','LINENAME']).sum()

In [20]:
# change columns name; AM ENTERS / AM EXITS
# remane AM columns: entries / exits
ts_11_2018_am_tot = ts_11_2018_am_tot.rename(columns={'ENTERS': 'AM_ENTERS'})
ts_11_2018_am_tot = ts_11_2018_am_tot.rename(columns={'EXITS' : 'AM_EXITS'})
ts_11_2018_am_tot.head()

AM_ENTERS  AM_EXITS
STATION       LINENAME                     
1 AV          L            2173.0    5427.0
103 ST        1            7888.0    1943.0
              6            8439.0    7932.0
              BC           4581.0    1183.0
103 ST-CORONA 7           22039.0    1903.0

In [21]:
# check total number of stations we have
ts_11_2018_am_tot.count()

AM_ENTERS    469
AM_EXITS     469
dtype: int64

In [22]:
# save as csv
ts_11_2018_am_tot.to_csv('ts_11_2018_am_tot.csv')

In [23]:
# Repeat for PM commute
# new data frame = focus on evening commute: 3:00pm AND 4:00pm AND 5:00pm AND 5:30pm AND 3:22pm
ts_11_2018_pm = ts_11_2018

In [24]:
# CORRECT way to isolate evening commuters!
ts_11_2018_pm = ts_11_2018_pm.loc[(ts_11_2018_pm["TIME"].dt.hour >= 15) & (ts_11_2018_pm["TIME"].dt.hour <= 18), :]
ts_11_2018_pm.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,TOT. ENTRIES,TOT. EXITS,WEEKDAY,ENTERS,EXITS
15,A002,R051,02-00-00,59 ST,NQR456W,BMT,2018-11-12,2019-08-27 15:00:00,REGULAR,6831812,2316785,Monday,242.0,72.0
21,A002,R051,02-00-00,59 ST,NQR456W,BMT,2018-11-13,2019-08-27 15:00:00,REGULAR,6833282,2317313,Tuesday,237.0,76.0
41,A002,R051,02-00-01,59 ST,NQR456W,BMT,2018-11-12,2019-08-27 15:00:00,REGULAR,6104595,1368087,Monday,158.0,40.0
47,A002,R051,02-00-01,59 ST,NQR456W,BMT,2018-11-13,2019-08-27 15:00:00,REGULAR,6105731,1368388,Tuesday,161.0,50.0
67,A002,R051,2/3/00,59 ST,NQR456W,BMT,2018-11-12,2019-08-27 15:00:00,REGULAR,1152801,4320461,Monday,68.0,341.0


In [28]:
# # SO MUCH MORE COMPLICATED - look at only 3pm AND 4pm data
# ts_11_2018_pm = ts_11_2018_pm.loc[(ts_11_2018_pm["TIME"]=="15:00:00") | (ts_11_2018_pm["TIME"]=="15:22:00") |
#                                   (ts_11_2018_pm["TIME"]=="15:30:00") | (ts_11_2018_pm["TIME"]=="15:47:18") |
#                                   (ts_11_2018_pm["TIME"]=="16:00:00") | (ts_11_2018_pm["TIME"]=="16:13:51") |
#                                   (ts_11_2018_pm["TIME"]=="16:34:12") | (ts_11_2018_pm["TIME"]=="16:34:13") |
#                                   (ts_11_2018_pm["TIME"]=="17:00:00") | (ts_11_2018_pm["TIME"]=="17:15:28") |
#                                   (ts_11_2018_pm["TIME"]=="17:30:00") | (ts_11_2018_pm["TIME"]=="18:00:00") |
#                                   (ts_11_2018_pm["TIME"]=="18:27:48") | (ts_11_2018_pm["TIME"]=="18:27:49") |
#                                   (ts_11_2018_pm["TIME"]=="19:00:00"), :]

In [25]:
# clean up data frame
ts_11_2018_pm = ts_11_2018_pm.drop(['C/A','UNIT','SCP','DIVISION','DESC','TOT. ENTRIES','TOT. EXITS'], axis=1)

In [26]:
# groupby station name and get total (not average)
ts_11_2018_pm_tot = ts_11_2018_pm.groupby(['STATION','LINENAME']).sum()

In [27]:
# change columns name; PM ENTERS / PM EXITS
ts_11_2018_pm_tot = ts_11_2018_pm_tot.rename(columns={'ENTERS': 'PM_ENTERS'})
ts_11_2018_pm_tot = ts_11_2018_pm_tot.rename(columns={'EXITS' : 'PM_EXITS'})
ts_11_2018_pm_tot.head()

PM_ENTERS  PM_EXITS
STATION       LINENAME                     
1 AV          L           17391.0   18195.0
103 ST        1           13537.0    6671.0
              6           16862.0   12606.0
              BC           4818.0    3202.0
103 ST-CORONA 7           14701.0    9735.0

In [28]:
# save as csv
ts_11_2018_pm_tot.to_csv('ts_11_2018_pm_tot.csv')

In [29]:
# merge AM and PM into one dataframe
commuters_11_2018 = pd.merge(ts_11_2018_am_tot, ts_11_2018_pm_tot, on=["STATION", "LINENAME"], how="outer")
commuters_11_2018.head()

AM_ENTERS  AM_EXITS  PM_ENTERS  PM_EXITS
STATION       LINENAME                                          
1 AV          L            2173.0    5427.0    17391.0   18195.0
103 ST        1            7888.0    1943.0    13537.0    6671.0
              6            8439.0    7932.0    16862.0   12606.0
              BC           4581.0    1183.0     4818.0    3202.0
103 ST-CORONA 7           22039.0    1903.0    14701.0    9735.0

In [30]:
# save as csv
commuters_11_2018.to_csv('commuters_11_2018.csv')

In [33]:
csv_path = 'commuters_11_2018.csv'
commuters_11_2018_2 = pd.read_csv(csv_path, encoding="utf-8")

commuters_11_2018_2.head()

,STATION,LINENAME,AM_ENTERS,AM_EXITS,PM_ENTERS,PM_EXITS
0,1 AV,L,2173.0,5427.0,17391.0,18195.0
1,103 ST,1,7888.0,1943.0,13537.0,6671.0
2,103 ST,6,8439.0,7932.0,16862.0,12606.0
3,103 ST,BC,4581.0,1183.0,4818.0,3202.0
4,103 ST-CORONA,7,22039.0,1903.0,14701.0,9735.0


In [44]:
# check Kew Gardens - TOO MANY PEOPLE
# kew_gardens = commuters_11_2018_2[commuters_11_2018_2['STATION'] == 'KEW GARDENS']
# kew_gardens